In [83]:
import pandas as pd
data = pd.read_csv('Sample_1.csv')


In [84]:
data.columns

Index(['p_id', 'age', 'female', 'ethnicity', 'grip', 'chest', 'e_anx_1',
       'e_anx_2', 'e_anx_3', 'e_anx_4', 'e_anx_5', 'e_anx_6', 'e_anx_7',
       'e_anx_8', 'e_anx_9', 'e_anx_10', 'e_dep_1', 'e_dep_2', 'e_dep_3',
       'e_dep_4', 'e_dep_5', 'e_dep_6', 'e_dep_7', 'e_dep_8', 'e_dep_9',
       'e_dep_10', 'e_fear_1', 'e_fear_2', 'e_fear_3', 'e_fear_4', 'e_fear_5',
       'e_fear_6', 'e_fear_7', 'e_fear_8', 'e_fear_9', 'e_fear_10', 'e_sen_1',
       'e_sen_2', 'e_sen_3', 'e_sen_4', 'e_sen_5', 'e_sen_6', 'e_sen_7',
       'e_sen_8', 'e_sen_9', 'e_sen_10'],
      dtype='object')

In [85]:
mean_anx = data[['e_anx_1', 'e_anx_2', 'e_anx_3','e_anx_4', 'e_anx_5', 'e_anx_6','e_anx_7', 'e_anx_8', 'e_anx_9', 'e_anx_10']].mean(axis=1)
mean_dep = data[['e_dep_1', 'e_dep_2', 'e_dep_3','e_dep_4', 'e_dep_5', 'e_dep_6','e_dep_7', 'e_dep_8', 'e_dep_9', 'e_dep_10']].mean(axis=1)
mean_fear = data[['e_fear_1', 'e_fear_2', 'e_fear_3','e_fear_4', 'e_fear_5', 'e_fear_6','e_fear_7', 'e_fear_8', 'e_fear_9', 'e_fear_10']].mean(axis=1)
mean_sen = data[['e_sen_1', 'e_sen_2', 'e_sen_3','e_sen_4', 'e_sen_5', 'e_sen_6','e_sen_7', 'e_sen_8', 'e_sen_9', 'e_sen_10']].mean(axis=1)

In [86]:
data['mean_anx']=mean_anx
data['mean_dep']=mean_dep
data['mean_fear']=mean_fear
data['mean_sen']=mean_sen

In [87]:
to_drop = ['e_anx_1', 'e_anx_2', 'e_anx_3','e_anx_4', 'e_anx_5', 'e_anx_6','e_anx_7', 'e_anx_8', 'e_anx_9', 'e_anx_10',\
          'e_dep_1', 'e_dep_2', 'e_dep_3','e_dep_4', 'e_dep_5', 'e_dep_6','e_dep_7', 'e_dep_8', 'e_dep_9', 'e_dep_10',\
          'e_fear_1', 'e_fear_2', 'e_fear_3','e_fear_4', 'e_fear_5', 'e_fear_6','e_fear_7', 'e_fear_8', 'e_fear_9', 'e_fear_10',\
          'e_sen_1', 'e_sen_2', 'e_sen_3','e_sen_4', 'e_sen_5', 'e_sen_6','e_sen_7', 'e_sen_8', 'e_sen_9', 'e_sen_10']
data.drop(columns=to_drop, inplace=True)

In [88]:
data.columns

Index(['p_id', 'age', 'female', 'ethnicity', 'grip', 'chest', 'mean_anx',
       'mean_dep', 'mean_fear', 'mean_sen'],
      dtype='object')

In [89]:
# Import necessary libraries
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [90]:
# Drop the rows with missing values
data = data.dropna()

# Split the dataset into features and target variable
X = data.drop(['mean_fear'], axis=1) # Drop fearfulness score as it is the target variable
y = data['mean_fear']

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#print(r)

In [91]:
# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [92]:
# Train the SVM model
svm_reg = SVR(kernel='linear', C=1)
svm_reg.fit(X_train, y_train)

SVR(C=1, kernel='linear')

In [93]:
# Predict the fearfulness score of the test set
y_pred = svm_reg.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Evaluate the model performance
print("Mean Squared Error:", mse)
print("Mean Absolute Error:", mae)
print("R^2:", r2)

Mean Squared Error: 0.3009528199370184
Mean Absolute Error: 0.4395422440575617
R^2: 0.05677939819155853


In [95]:
#SVM classification

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, f1_score

# Convert fearfulness scores into three categories
threshold1 = data['mean_fear'].quantile(0.33)
threshold2 = data['mean_fear'].quantile(0.66)
data['fearfulness_category'] = pd.cut(data['mean_fear'], bins=[-float('inf'), threshold1, threshold2, float('inf')], labels=['low', 'medium', 'high'])

# Drop the rows with missing values
data = data.dropna()

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data.drop(['mean_fear', 'fearfulness_category'], axis=1), data['fearfulness_category'], test_size=0.2, random_state=42)

# Train the SVM classifier
svm = SVC(kernel='linear')
svm.fit(X_train, y_train)

# Make predictions on the testing set
y_pred = svm.predict(X_test)

# Evaluate the performance of the classifier
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='weighted')

print(f'Accuracy: {accuracy}')
print(f'F1-score: {f1}')

Accuracy: 0.3142857142857143
F1-score: 0.30727513227513226
